In [1]:
# Importing relevant libraries

import numpy as np
import pandas as pd
import matplotlib
import pymysql

# Loading the Data

In [2]:
# Establishing connection to the db

try:
    conn = pymysql.connect(
        database = 'sakila', 
        user = 'root', 
        password = '365pass')
    
except:
    print('Error connecting to the databse')
    
else:
    print('Connected to the Sakila database')


Connected to the Sakila database


In [5]:
# Query tested on MySQL to retrive data for analysis
myQuery = '''
SELECT 
	film.title, film.rental_rate, film.replacement_cost, film.rating, 
    store.store_id,
    address.district, city.city, country.country,
    rental.rental_id, DATE(rental.rental_date) AS rental_date, DATE(rental.return_date) AS return_date,
    datediff(DATE(rental.return_date), DATE(rental.rental_date)) AS rental_actual_duration,
    payment.amount,
    customer.first_name, customer.last_name, customer.email
FROM 
	inventory
		INNER JOIN
    film ON inventory.film_id = film.film_id
		INNER JOIN
	store ON inventory.store_id = store.store_id
		INNER JOIN
	rental ON inventory.inventory_id = rental.inventory_id
		INNER JOIN
	address ON store.address_id = address.address_id
		INNER JOIN
	city ON address.city_id = city.city_id
		INNER JOIN
	country ON city.country_id = country.country_id
		INNER JOIN
	customer ON rental.customer_id = customer.customer_id
        INNER JOIN
    payment ON rental.rental_id = payment.rental_id;'''

In [6]:
# Executing SQL query
cursor = conn.cursor() 
cursor.execute(myQuery)
rows = cursor.fetchall()

# Converting the queried rows to a DataFrame
# DataFrames are more efficient and easier to make analysis

rows = np.asarray(rows)
df = pd.DataFrame(rows, columns = (['Film_Title', 'Film_Rental_Rate', 'Film_Replace_Cost', 'Film_Rating', 
                                    'Store_ID',
                                    'District', 'City', 'Country',
                                    'Rental_ID', 'Rental_Date', 'Rental_Return_Date',
                                    'Rental_Duration',
                                    'Payment_Amount',
                                    'Customer_First_Name', 'Customer_Last_Name', 'Customer_Email']))

# Closing the connection
cursor.close()
conn.close()

# OVERVIEW OF THE DATA

In [7]:
df.shape

(16044, 16)

DF has: 16044 rows and 15 columns

In [8]:
df.head()

,Film_Title,Film_Rental_Rate,Film_Replace_Cost,Film_Rating,Store_ID,District,City,Country,Rental_ID,Rental_Date,Rental_Return_Date,Rental_Duration,Payment_Amount,Customer_First_Name,Customer_Last_Name,Customer_Email
0,ACADEMY DINOSAUR,0.99,20.99,PG,1,Alberta,Lethbridge,Canada,4863,2005-07-08,2005-07-11,3,0.99,JOEL,FRANCISCO,JOEL.FRANCISCO@sakilacustomer.org
1,ACADEMY DINOSAUR,0.99,20.99,PG,1,Alberta,Lethbridge,Canada,11433,2005-08-02,2005-08-11,9,3.99,GABRIEL,HARDER,GABRIEL.HARDER@sakilacustomer.org
2,ACADEMY DINOSAUR,0.99,20.99,PG,1,Alberta,Lethbridge,Canada,14714,2005-08-21,2005-08-30,9,3.99,DIANNE,SHELTON,DIANNE.SHELTON@sakilacustomer.org
3,ACADEMY DINOSAUR,0.99,20.99,PG,1,Alberta,Lethbridge,Canada,972,2005-05-30,2005-06-06,7,1.99,NORMAN,CURRIER,NORMAN.CURRIER@sakilacustomer.org
4,ACADEMY DINOSAUR,0.99,20.99,PG,1,Alberta,Lethbridge,Canada,2117,2005-06-17,2005-06-23,6,0.99,BEATRICE,ARNOLD,BEATRICE.ARNOLD@sakilacustomer.org


In [12]:
df.describe()

,Film_Title,Film_Rental_Rate,Film_Replace_Cost,Film_Rating,Store_ID,District,City,Country,Rental_ID,Rental_Date,Rental_Return_Date,Rental_Duration,Payment_Amount,Customer_First_Name,Customer_Last_Name,Customer_Email
count,16044,16044,16044,16044,16044,16044,16044,16044,16044,16044,15861,15861,16044,16044,16044,16044
unique,958,3,21,5,2,2,2,2,16044,41,84,11,19,591,599,599
top,BUCKET BROTHERHOOD,0.99,22.99,PG-13,2,QLD,Woodridge,Australia,4863,2005-07-31,2005-08-04,7,4.99,MARION,HUNT,ELEANOR.HUNT@sakilacustomer.org
freq,34,5652,1044,3585,8121,8121,8121,8121,1,679,534,1821,3789,68,46,46


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16044 entries, 0 to 16043
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Film_Title           16044 non-null  object
 1   Film_Rental_Rate     16044 non-null  object
 2   Film_Replace_Cost    16044 non-null  object
 3   Film_Rating          16044 non-null  object
 4   Store_ID             16044 non-null  object
 5   District             16044 non-null  object
 6   City                 16044 non-null  object
 7   Country              16044 non-null  object
 8   Rental_ID            16044 non-null  object
 9   Rental_Date          16044 non-null  object
 10  Rental_Return_Date   15861 non-null  object
 11  Rental_Duration      15861 non-null  object
 12  Payment_Amount       16044 non-null  object
 13  Customer_First_Name  16044 non-null  object
 14  Customer_Last_Name   16044 non-null  object
 15  Customer_Email       16044 non-null  object
dtypes: o

We can see that Return_Date has 15861 Non-Null values. Let's verify how many Nulls there are.

In [14]:
count=0
for i in df['Rental_Return_Date']:
    if(i is None):
        count+=1
print(count)

183


In [15]:
15861+count

16044

So, we have 183 Null values that match with the 16044 when summed with 15861 Non-Null values.

This evidence means that 183 times people did not return the movies DVD. 

# Analyzing Numerical Field

Let's analyze the Payment_Amount

In [21]:
type(df['Payment_Amount'])

pandas.core.series.Series

In [17]:
df['Payment_Amount'].describe()

count     16044
unique       19
top        4.99
freq       3789
Name: Payment_Amount, dtype: object

In [18]:
df['Payment_Amount'].mean()

4.2013562702567935

In [19]:
df['Payment_Amount'].median()

3.99